In [172]:
# --- IMPORT STATEMENTS ---

import json
import pandas as pd
from gurobipy import Model, GRB, quicksum, GurobiError
from collections import deque

# -------------------------

In [173]:
# --- CIV 6 DISTRICT PLACEMENT OPTIMIZATION MODEL ---
# Input sets / Data Initialization

# parse input data from civ_map_data.json into a pandas dataframe
try:
    with open('civ_map_data.json') as f:
        data = json.load(f)
    df = pd.DataFrame(data['tiles'])
    print("Loaded civ_map_data.json successfully.")
    # print(df.head())
except FileNotFoundError: # ERROR CHECKING
    print("ERROR: civ_map_data.json not found. Please ensure it's in the correct directory.")
except Exception as e:
    print(f"ERROR: Could not load or parse civ_map_data.json: {e}")


# ASSUMPTION: the city center is selected at the tile with the highest normalized score.
# Ensure the dataframe 'df' and the column 'normalized_score' exist
if 'normalized_score' in df.columns:
    best_tile = df.loc[df['normalized_score'].idxmax()]

    cc_x = int(best_tile['x']) # Ensure integer coords
    cc_y = int(best_tile['y'])

    city_center_coords = (cc_x, cc_y)
    print(f"City Center coordinates selected: {city_center_coords}")
else:
    print("ERROR: 'normalized_score' column not found in DataFrame. Cannot determine City Center.")
    # Handle error - e.g., set a default CC or exit
    # city_center_coords = (some_default_x, some_default_y)
    # exit()

Loaded civ_map_data.json successfully.
City Center coordinates selected: (56, 25)


In [174]:
# --- Constants and Data Structures ---

# Set of Districts to consider
ALL_DISTRICTS = [
    "Campus", "Holy Site", "Harbor", "Government Plaza", # Ensured GP is present
    "Theater Square", "Entertainment Complex", "Commercial Hub",
    "Industrial Zone", "Aqueduct", "Water Park", "Dam", "Canal"
    # Add others if needed, e.g., Diplomatic Quarter, Preserve
]

# Unique Districts (usually matches ALL_DISTRICTS except maybe Neighborhood)
DISTRICTS_UNIQUE = [
     "Campus", "Holy Site", "Harbor", "Government Plaza", # Ensured GP is present
     "Theater Square", "Entertainment Complex", "Commercial Hub",
     "Industrial Zone", "Aqueduct", "Water Park", "Dam", "Canal"
     # Add others if needed
]

# Mutually Exclusive Sets
DISTRICTS_MUTUALLY_EXCLUSIVE = [
    {"Entertainment Complex", "Water Park"}
]

DISTRICT_BASE_VALUE = {
    "Campus": 2.0,
    "Industrial Zone": 2.0,
    "Commercial Hub": 2.0,
    "Theater Square": 2.0,
    "Holy Site": 2.0,
    "Harbor": 2.0,
    "Government Plaza": 3.0 # Higher value since it provides adjacency to others
    # Add values for other districts
}

# Features relevant for adjacency or placement
FEATURES = {
    "Mountain": ["TERRAIN_GRASS_MOUNTAIN", "TERRAIN_DESERT_MOUNTAIN", "TERRAIN_PLAINS_MOUNTAIN", "TERRAIN_TUNDRA_MOUNTAIN", "TERRAIN_SNOW_MOUNTAIN"],
    "Coast": ["TERRAIN_COAST"],
    "River": ["River"], # Placeholder for river logic
    "Lake": ["TERRAIN_LAKE"],
    "Oasis": ["TERRAIN_DESERT_OASIS"],
    "Geothermal Fissure": ["FEATURE_GEOTHERMAL_FISSURE"],
    "Reef": ["FEATURE_REEF"],
    "Woods": ["FEATURE_WOODS"],
    "Rainforest": ["FEATURE_JUNGLE"],
    "Marsh": ["FEATURE_MARSH"],
    "Floodplains": ["FEATURE_FLOODPLAINS", "FEATURE_FLOODPLAINS_GRASSLAND", "FEATURE_FLOODPLAINS_PLAINS"], # Added Floodplains
    "NaturalWonder": ["NaturalWonder"], # Placeholder if you add specific NW logic
    "Wonder": ["Wonder"], # Placeholder for Built Wonders if treated as features
    "Strategic": ["StrategicResource"], # Placeholder
    "Luxury": ["LuxuryResource"] # Placeholder
}

# Terrain Adjacency Bonuses
TERRAIN_BONUS_VALUE = {
    ("Campus", "Mountain"): 1, ("Campus", "Reef"): 2, ("Campus", "Geothermal Fissure"): 2,
    ("Campus", "Rainforest"): 0.5,
    ("Holy Site", "Mountain"): 1, ("Holy Site", "Woods"): 0.5,
    ("Holy Site", "Natural Wonder"): 2, # Assumes "NaturalWonder" feature exists
    ("Commercial Hub", "River"): 2,     # Handled by river logic later
    ("Theater Square", "Wonder"): 2,      # Assumes "Wonder" feature exists for built wonders
    ("Industrial Zone", "Strategic"): 1, # Example if needed
    ("Aqueduct", "Geothermal Fissure"): 2, # Example bonus TO aqueduct
    # Note: River bonus for CH is handled separately based on river edge data
}
# Ensure "River" key exists if used for CH bonus calculation later
RIVER_FEATURE_KEY = "River"

# City Center Adjacency Bonuses
CC_BONUS_VALUE = { "Harbor": 2 } # Default others to 0 later

# District-to-District Adjacency Bonuses
# Format: (District Receiving Bonus, Adjacent District/Feature Providing Bonus): Bonus Value
DISTRICT_PAIR_BONUS = {
    # --- Original Pairs ---
    ('Commercial Hub', 'Harbor'): 2,
    
    # ('Theater Square', 'Wonder'): 2, # Covered by TERRAIN_BONUS_VALUE if Wonder is a Feature
    ('Theater Square', 'Entertainment Complex'): 1,
    ('Theater Square', 'Water Park'): 1,
    ('Industrial Zone', 'Aqueduct'): 2,
    ('Industrial Zone', 'Dam'): 2,
    ('Industrial Zone', 'Canal'): 2,
    # --- Government Plaza Provided Bonuses ---
    ('Campus',          'Government Plaza'): 1, # <--- Set back to 1 (or 10 for testing)
    ('Holy Site',       'Government Plaza'): 1,
    ('Harbor',          'Government Plaza'): 1,
    ('Commercial Hub',  'Government Plaza'): 1,
    ('Industrial Zone', 'Government Plaza'): 1,
    ('Theater Square',  'Government Plaza'): 1,
}

# Generate standard district adjacency bonuses (0.5 for any district pair)
def generate_standard_district_adjacencies():
    standard_pairs = {}
    # Districts that receive adjacency from any district
    receiving_districts = ["Campus", "Holy Site", "Theater Square", "Industrial Zone", "Commercial Hub"]
    
    for receiving in receiving_districts:
        for giving in ALL_DISTRICTS:
            # Skip if it's the same district or if a specific bonus already exists
            if receiving != giving and (receiving, giving) not in DISTRICT_PAIR_BONUS:
                standard_pairs[(receiving, giving)] = 0.5
    
    return standard_pairs

# Extend DISTRICT_PAIR_BONUS with standard adjacencies
STANDARD_DISTRICT_ADJACENCIES = generate_standard_district_adjacencies()
DISTRICT_PAIR_BONUS.update(STANDARD_DISTRICT_ADJACENCIES)
print(f"Added {len(STANDARD_DISTRICT_ADJACENCIES)} standard district adjacency pairs")

# Default CC Bonus to 0 for districts not listed
for dist in ALL_DISTRICTS: CC_BONUS_VALUE.setdefault(dist, 0)

# Other Constants
RADIUS = 3
MAX_ADJACENT_TILES = 6 # Max neighbors in hex grid

IMPASSABLE_TERRAIN = {
    "TERRAIN_OCEAN", "TERRAIN_GRASS_MOUNTAIN", "TERRAIN_DESERT_MOUNTAIN", "TERRAIN_PLAINS_MOUNTAIN",
    "TERRAIN_TUNDRA_MOUNTAIN", "TERRAIN_SNOW_MOUNTAIN", "TERRAIN_LAKE",
}
IMPASSABLE_FEATURES = { "FEATURE_ICE" }


# Map yields to their primary districts
YIELD_TO_DISTRICT = {
    "science": "Campus",
    "culture": "Theater Square",
    "production": "Industrial Zone",
    "gold": "Commercial Hub",
    "faith": "Holy Site",
    "food": "None",  # No direct district, but affects overall strategy
}

# Create priority profiles with district value multipliers
def create_priority_profile(primary_yield, boost_factor=2.0):
    """Creates a district priority profile based on primary yield"""
    profile = {}
    
    # Set baseline values for all districts
    for district in ALL_DISTRICTS:
        if district == YIELD_TO_DISTRICT.get(primary_yield):
            # Primary district gets boosted value
            profile[district] = DISTRICT_BASE_VALUE.get(district, 1.0) * boost_factor
        elif district == "Industrial Zone" and primary_yield != "production":
            # Production helps everything, so IZ still gets a small boost
            profile[district] = DISTRICT_BASE_VALUE.get(district, 1.0) * 1.2
        elif district == "Government Plaza":
            # GP always valuable for adjacency bonuses
            profile[district] = DISTRICT_BASE_VALUE.get(district, 3.0)
        else:
            # Other districts get normal values
            profile[district] = DISTRICT_BASE_VALUE.get(district, 1.0)
            
    return profile

print("Constants and data structures defined.")
# print("Checking DISTRICT_PAIR_BONUS for ('Campus', 'Government Plaza'):", DISTRICT_PAIR_BONUS.get(('Campus', 'Government Plaza'), 'MISSING')) # Debug check

Added 44 standard district adjacency pairs
Constants and data structures defined.


In [175]:
# --- Helper Functions ---

def get_adjacent_tiles(x, y, all_map_coords_set):
    """ Gets coordinates of adjacent tiles based on 'odd-r' flat-top logic. """
    x = int(x)
    y = int(y)
    direct_offsets = []
    is_even_row = y % 2 == 0

    if is_even_row: # EVEN rows (y=0, 2, 4...)
        direct_offsets = [ [+1,  0], [+1, +1], [ 0, +1], [-1,  0], [ 0, -1], [+1, -1] ]
    else: # ODD rows (y=1, 3, 5...)
        direct_offsets = [ [+1,  0], [ 0, +1], [-1, +1], [-1,  0], [-1, -1], [ 0, -1] ]

    adjacent = set()
    for dx, dy in direct_offsets:
        neighbor_coords = (x + dx, y + dy)
        if neighbor_coords in all_map_coords_set:
            adjacent.add(neighbor_coords)
    return adjacent

def get_tiles_in_radius_bfs(start_coords, max_rings, all_map_coords_set):
    """ Finds all tiles within max_rings steps from start_coords using BFS. """
    start_x, start_y = int(start_coords[0]), int(start_coords[1])
    start_coords_int = (start_x, start_y)

    if start_coords_int not in all_map_coords_set:
        print(f"Warning: Start coordinates {start_coords_int} not in map data.")
        return set()
    if max_rings <= 0: return {start_coords_int}

    queue = deque([(start_coords_int, 0)])
    visited = {start_coords_int}
    reachable_tiles = {start_coords_int}

    while queue:
        current_coords, current_dist = queue.popleft()
        if current_dist < max_rings:
            neighbors = get_adjacent_tiles(current_coords[0], current_coords[1], all_map_coords_set)
            for neighbor in neighbors:
                if neighbor not in visited:
                    visited.add(neighbor)
                    reachable_tiles.add(neighbor)
                    queue.append((neighbor, current_dist + 1))
    return reachable_tiles

print("Helper functions defined.")

Helper functions defined.


In [176]:
# --- Pre-calculation Steps ---

# Set of all valid (x, y) coordinates from the DataFrame
all_map_coords = set((int(coord[0]), int(coord[1])) for coord in df[['x', 'y']].values)

# Tiles: Set of tiles within the defined radius using BFS
tiles_in_radius = get_tiles_in_radius_bfs(city_center_coords, RADIUS, all_map_coords)
print(f"Total tiles in radius {RADIUS}: {len(tiles_in_radius)}")
if not tiles_in_radius:
     print("ERROR: No tiles found within radius. Check CC coords and map data.")
     # exit()

# Mapping from (x, y) tuple to the tile's data (only for tiles in radius)
tile_data = {
    (int(row['x']), int(row['y'])): row for index, row in df.iterrows()
    if (int(row['x']), int(row['y'])) in tiles_in_radius
}

# Tiles_Workable: Subset of tiles_in_radius where districts can potentially be built
tiles_workable = set()
coast_terrain_values = FEATURES.get("Coast", ["TERRAIN_COAST"]) # Get list of coast terrain types
for coords in tiles_in_radius:
    if coords == city_center_coords: continue
    if coords not in tile_data: continue

    tile_info = tile_data[coords]
    terrain = tile_info['terrain']
    feature = tile_info.get('feature') # Use .get for features which might be null

    is_impassable_terrain = terrain in IMPASSABLE_TERRAIN
    is_impassable_feature = feature is not None and feature in IMPASSABLE_FEATURES
    is_impassable = is_impassable_terrain or is_impassable_feature

    # Allow placement on Coast tiles (for Harbor), otherwise check impassable
    if terrain in coast_terrain_values or not is_impassable:
         # Add further checks if needed (e.g., some features block certain districts)
         tiles_workable.add(coords)

print(f"Workable tiles in radius: {len(tiles_workable)}")

# Adjacency Map: Maps (x, y) -> set of adjacent (x', y') within the radius
adjacency_map = {
    coords: get_adjacent_tiles(coords[0], coords[1], all_map_coords).intersection(tiles_in_radius)
    for coords in tiles_in_radius
}

# Features on Tile Map & River Adjacency Map
features_on_tile = {}
has_river_edge = {} # Maps (x, y) -> boolean, True if tile itself has river edge data

for coords in tiles_in_radius:
    if coords not in tile_data: continue
    tile_info = tile_data[coords]
    tile_features = set() # Features relevant for providing bonuses (Mountain, Reef etc)

    terrain = tile_info['terrain']
    feature = tile_info.get('feature')
    resource = tile_info.get('resource')
    res_type = tile_info.get('resourcetype', '') # Default to empty string
    is_nw = tile_info.get('isNaturalWonder', False) # Assumes boolean flag exists
    # Add check for built Wonders if needed e.g., tile_info.get('isWonderBuilt', False)

    # Map terrain/feature names from FEATURES constant (excluding River, NW, Wonder, Resources)
    # These provide adjacency bonus
    for f_name, f_data_vals in FEATURES.items():
        # Handle bonus-providing features based on *their* terrain/feature type
         if f_name not in {"River", "NaturalWonder", "Wonder", "Strategic", "Luxury", "Floodplains", "Coast", "Lake", "Oasis"}: # Exclude types handled differently or non-bonus giving terrain itself
              if terrain in f_data_vals or (feature is not None and feature in f_data_vals):
                   tile_features.add(f_name)

    # Add special feature flags to the set for the tile itself
    if is_nw: tile_features.add("NaturalWonder")
    # if tile_info.get('isWonderBuilt', False): tile_features.add("Wonder") # Add if you track built wonders this way
    if resource:
        if res_type is not None:
            if res_type.lower() == 'strategic': tile_features.add("Strategic")
            elif res_type.lower() == 'luxury': tile_features.add("Luxury")

    features_on_tile[coords] = tile_features

    # River Edge Check (Crucial for River Bonus)
    # Assumes 'rivers' field exists and is non-empty string/list/dict if river is present
    if pd.notna(tile_info.get('rivers')) and tile_info.get('rivers') not in ["", None, [], {}]:
        has_river_edge[coords] = True
    else:
        has_river_edge[coords] = False

print(f"Tiles with river edges identified: {sum(has_river_edge.values())}")
# print({k: v for k, v in has_river_edge.items() if v}) # Optional: Print tiles with rivers

Total tiles in radius 3: 37
Workable tiles in radius: 29
Tiles with river edges identified: 10


In [177]:
# --- Pre-calculation of Boolean Maps & Adjacency ---

# isAdjacentToCC: Check adjacency for workable tiles
is_adjacent_to_cc = {
    coords: city_center_coords in adjacency_map.get(coords, set())
    for coords in tiles_workable
}

# isAdjacentToFreshwater: Check adjacency for workable tiles
# Freshwater sources: River, Lake, Oasis, Mountain (for Aqueduct)
freshwater_feature_keys = {"River", "Lake", "Oasis", "Mountain"}
is_adjacent_to_freshwater = {}
for coords in tiles_workable:
    adj_fresh = False
    for adj_coords in adjacency_map.get(coords, set()):
         # Check if adjacent tile IS a freshwater source (Lake/Oasis TILE or Mountain TILE)
         # or if the current tile has a river edge crossing to the neighbor
         if adj_coords in tile_data:
              adj_tile_info = tile_data[adj_coords]
              adj_terrain = adj_tile_info['terrain']
              adj_feature = adj_tile_info.get('feature')

              # Check Mountain, Lake, Oasis terrain/feature on adjacent tile
              if any(f_key in FEATURES and (adj_terrain in FEATURES[f_key] or (adj_feature is not None and adj_feature in FEATURES[f_key])) for f_key in {"Mountain", "Lake", "Oasis"}):
                   adj_fresh = True
                   break
              # Check if THIS tile has a river edge (more reliable than checking if neighbor IS river)
              if has_river_edge.get(coords, False):
                   adj_fresh = True
                   break # Found freshwater source (river edge)
    is_adjacent_to_freshwater[coords] = adj_fresh


# isCoast: Check terrain for workable tiles
coast_terrain_values = FEATURES.get("Coast", ["TERRAIN_COAST"])
is_coast = {
    coords: tile_data[coords]['terrain'] in coast_terrain_values
    for coords in tiles_workable if coords in tile_data
}

# isAdjacentToLand: Check adjacency for workable tiles (primarily for Harbor)
# Define land terrains (anything not Ocean, Coast, Lake - adjust if needed)
land_terrains = { t for t in df['terrain'].unique() if t not in {"TERRAIN_OCEAN", "TERRAIN_COAST", "TERRAIN_LAKE"} }
is_adjacent_to_land = {}
for coords in tiles_workable:
    # Only relevant if the tile itself is Coast
    if is_coast.get(coords, False):
        adj_land = False
        for adj_coords in adjacency_map.get(coords, set()):
             if adj_coords in tile_data and tile_data[adj_coords]['terrain'] in land_terrains:
                  adj_land = True
                  break
        is_adjacent_to_land[coords] = adj_land
    else:
        is_adjacent_to_land[coords] = False # Not relevant if the tile itself isn't Coast

# isFloodplains: Check feature for workable tiles (for Dam)
floodplains_feature_values = FEATURES.get("Floodplains", [])
is_floodplains = {
     coords: tile_data[coords].get('feature') in floodplains_feature_values
     for coords in tiles_workable if coords in tile_data
}

print("Pre-calculation of boolean maps finished.")

Pre-calculation of boolean maps finished.


In [178]:
# --- Initialize Gurobi Model ---
model = Model("Civ6_District_Optimization")
model.setParam('OutputFlag', 1) # Set to 1 for solver output, 0 to suppress
print("Gurobi model initialized.")

Set parameter OutputFlag to value 1
Gurobi model initialized.


In [179]:
# --- Define Main Decision Variables: Build(x, y, d) ---
build_vars = {}
print("Creating main 'Build' variables...")
for coords in tiles_workable:
    for d in ALL_DISTRICTS:
        var_name = f"Build[{coords[0]},{coords[1]},{d}]"
        build_vars[coords, d] = model.addVar(vtype=GRB.BINARY, name=var_name)

# Integrate new variables immediately (good practice)
model.update()
print(f"Gurobi 'Build' variables defined ({len(build_vars)} variables).")

Creating main 'Build' variables...
Gurobi 'Build' variables defined (348 variables).


In [180]:
# --- Define Auxiliary Variables for District Adjacency ---
adj_district_vars = {} # Format: adj_district_vars[coords_d1, d1, d2] = variable A

print("Defining auxiliary adjacency variables...")
# Create variables only for pairs defined in DISTRICT_PAIR_BONUS where d1 receives bonus from d2
for (d1, d2), bonus_value in DISTRICT_PAIR_BONUS.items():
    if bonus_value > 0:
        # Check if d2 is a district type or a feature like 'Wonder'
        is_d2_district = d2 in ALL_DISTRICTS
        is_d2_feature = d2 in FEATURES # e.g., 'Wonder' if handled as feature

        for coords_d1 in tiles_workable:
            # Ensure the primary district (d1) variable exists for this tile
            if (coords_d1, d1) in build_vars:
                 # Create the aux variable: AdjDist_x_y_d1_from_d2 = 1 if d1 at (x,y) gets bonus from adjacent d2
                 var_name = f"AdjDist_{coords_d1[0]}_{coords_d1[1]}_{d1}_from_{d2}"
                 # Use tuple key: (coords_tuple, d1_string, d2_string)
                 current_key = (coords_d1, d1, d2) # Define the key explicitly
                 adj_district_vars[current_key] = model.addVar(vtype=GRB.BINARY, name=var_name)

                 # --- Add specific debug print for variable creation ---
                 if coords_d1 == (51, 16) and d1 == 'Campus' and d2 == 'Government Plaza':
                      print(f"\n*** DEBUG: SUCCESSFULLY CREATED aux var for key {current_key} with name {var_name} ***\n")
                 # --- End debug print ---


# Integrate new variables
model.update()
print(f"Defined {len(adj_district_vars)} auxiliary adjacency variables.")
# Check if the specific key exists AFTER the loop
check_key = ((51, 16), 'Campus', 'Government Plaza')
if check_key in adj_district_vars:
    print(f"DEBUG: Key {check_key} EXISTS in adj_district_vars after creation loop.")
else:
    print(f"*** CRITICAL WARNING: Key {check_key} DOES NOT EXIST in adj_district_vars after creation loop! ***")

Defining auxiliary adjacency variables...
Defined 1624 auxiliary adjacency variables.
*** CRITICAL WARNING: Key ((51, 16), 'Campus', 'Government Plaza') DOES NOT EXIST in adj_district_vars after creation loop! ***


In [181]:
# --- Add Linking Constraints for Auxiliary Adjacency Variables ---
# Links A = adj_district_vars[coords_d1, d1, d2]
# To B1 = build_vars[coords_d1, d1]
# And S2 = Sum over adjacent tiles (adj_coords) of Build(adj_coords, d2) or HasFeature(adj_coords, d2)

print("Adding linking constraints for adjacency variables...")
constraints_added_count = 0

for (coords_d1, d1, d2), A in adj_district_vars.items():
    # Get the B1 variable (Build(d1) at coords_d1)
    if (coords_d1, d1) not in build_vars: continue # Should not happen based on creation logic, but safe check
    B1 = build_vars[coords_d1, d1]

    # Calculate S2 = Sum of adjacent triggers (d2)
    adjacent_tiles = adjacency_map.get(coords_d1, set())
    sum_expr_S2 = None # Initialize S2 expression

    # Determine if d2 is a District or a Feature triggering the bonus
    if d2 in ALL_DISTRICTS:
         # S2 is the sum of Build(adj_coords, d2) for adjacent workable tiles
         # Use a list comprehension inside quicksum for clarity if needed
         s2_terms = [build_vars.get((adj_coords, d2), 0)
                     for adj_coords in adjacent_tiles
                     if adj_coords in tiles_workable and (adj_coords, d2) in build_vars] # Ensure var exists
         sum_expr_S2 = quicksum(s2_terms)

    elif d2 in FEATURES: # Check if d2 is a key in FEATURES dict (e.g., "Wonder")
         # S2 is the sum of indicator constants (1 if feature present, 0 otherwise)
         sum_expr_S2 = quicksum(1
                                for adj_coords in adjacent_tiles
                                if adj_coords in features_on_tile and d2 in features_on_tile[adj_coords])
    else:
         # Skip if d2 is not a district or known feature key
         continue

    # --- DEBUG PRINT for S2 expression (Now Included) ---
    if d1 == 'Campus' and d2 == 'Government Plaza' and coords_d1 == (51, 16): # Check for specific Campus location
         print(f"\n*** DEBUG constraint S2 check for Campus at {coords_d1} from GP: ***")
         try:
              # Check if S2 is a valid Gurobi expression type
              if isinstance(sum_expr_S2, (int, float, type(None))): # Check if it resolved to a constant or None
                   print(f"  S2 Expression evaluated to constant/None: {sum_expr_S2}")
              else: # Assume it's a Gurobi LinExpr
                   print(f"  S2 Expression (Gurobi): {sum_expr_S2}")

              # Check components manually to see which vars S2 depends on
              terms = []
              s2_term_vars_found = False
              for adj_coords_debug in adjacency_map.get(coords_d1, set()):
                    if adj_coords_debug in tiles_workable:
                         key_debug = (adj_coords_debug, 'Government Plaza')
                         if key_debug in build_vars:
                              terms.append(build_vars[key_debug].VarName)
                              s2_term_vars_found = True
              print(f"  S2 depends on GP Build Vars on workable adjacent tiles: {terms if s2_term_vars_found else '*** NONE FOUND - THIS IS LIKELY THE PROBLEM ***'}")
         except Exception as e:
              print(f"  Error printing S2 expression/components: {e}")
         print("*** End S2 Check ***\n")
    # --- End S2 Debug Print ---


    # --- Constraints linking A to B1 and S2: A = 1 iff B1=1 and S2>=1 ---
    constr_name_base = f"LinkAdj_{coords_d1[0]}_{coords_d1[1]}_{d1}_{d2}"

    # Check if sum_expr_S2 is valid before adding constraints using it
    if sum_expr_S2 is not None:
         # 1. A must be 0 or 1
         model.addConstr(A <= 1)
          
         # 2. A can only be 1 if B1 is 1
         model.addConstr(A <= B1)
          
         # 3. Create indicator for adjacency
         has_adj = model.addVar(vtype=GRB.BINARY, name=f"{constr_name_base}_HasAdj")
         model.addConstr(sum_expr_S2 <= MAX_ADJACENT_TILES * has_adj)
         model.addConstr(has_adj * 0.1 <= sum_expr_S2)
          
         # 4. A can only be 1 if there's adjacency
         model.addConstr(A <= has_adj)
          
         # 5. A must be 1 if both B1 and has_adj are 1
         model.addConstr(A >= B1 + has_adj - 1)

     # else:
     #      print(f"Warning: sum_expr_S2 was None for {constr_name_base}. Constraints not added") # Optional


print(f"Added {constraints_added_count} linking constraints for adjacency.")

Adding linking constraints for adjacency variables...
Added 0 linking constraints for adjacency.


In [182]:
# --- Define Objective Function ---

print("Calculating objective function coefficients...")

# Calculate base coefficients from Terrain, Features (non-district), CC, River
obj_coeffs = {}
for coords in tiles_workable:
    for d in ALL_DISTRICTS:
        # Sum terrain/feature bonuses from adjacent tiles (non-River)
        terrain_feature_bonus = 0
        for adj_coords in adjacency_map.get(coords, set()):
             if adj_coords in features_on_tile:
                  for f in features_on_tile.get(adj_coords, set()):
                       # Look up bonus value, ensuring f isn't the River key used for edge logic
                       if f != RIVER_FEATURE_KEY:
                           terrain_feature_bonus += TERRAIN_BONUS_VALUE.get((d, f), 0)

        # Add City Center bonus if adjacent
        cc_bonus = CC_BONUS_VALUE.get(d, 0) if is_adjacent_to_cc.get(coords, False) else 0

        # Add River Adjacency Bonus if *this tile* has a river edge
        river_bonus = 0
        if has_river_edge.get(coords, False):
            river_bonus = TERRAIN_BONUS_VALUE.get((d, RIVER_FEATURE_KEY), 0)

        obj_coeffs[coords, d] = terrain_feature_bonus + cc_bonus + river_bonus

print("Calculating base objective expression...")
# Base objective part from terrain, CC, river bonuses
base_objective_expr = quicksum(
    build_vars[key] * obj_coeffs.get(key, 0)
    for key in build_vars.keys() # Iterate through all defined build variables
)

print("Calculating district adjacency bonus contribution...")
# District adjacency bonus part using the auxiliary variables
district_adj_bonus_expr_terms = []
processed_obj_keys = set() # Keep track of keys processed in this loop

for key, aux_var in adj_district_vars.items():
    # Ensure key is the expected tuple format (coords_tuple, d1_string, d2_string)
    if not (isinstance(key, tuple) and len(key) == 3):
         print(f"WARNING: Unexpected key format in adj_district_vars: {key}")
         continue
    coords_d1, d1, d2 = key # Unpack the key
    processed_obj_keys.add(key) # Mark this key as processed

    bonus = DISTRICT_PAIR_BONUS.get((d1, d2), 0)
    if bonus > 0:
        term = aux_var * bonus
        district_adj_bonus_expr_terms.append(term)

        # --- Add specific debug print for GP bonus ---
        if coords_d1 == (51, 16) and d1 == 'Campus' and d2 == 'Government Plaza':
            print(f"\n*** DEBUG: FOUND AND ADDING objective term for {aux_var.VarName} * {bonus} (GP->Campus Bonus) ***\n")
        # --- End debug print ---

# Check if the specific key we care about was missed in the loop
check_key_obj = ((51, 16), 'Campus', 'Government Plaza')
if check_key_obj in adj_district_vars and check_key_obj not in processed_obj_keys:
     print(f"*** CRITICAL WARNING: Key {check_key_obj} exists in adj_district_vars but WAS NOT processed in objective loop! ***")


# District base value contribution 
district_base_value_expr = quicksum(
    build_vars[key] * DISTRICT_BASE_VALUE.get(key[1], 0.0)  # key[1] is the district type
    for key in build_vars.keys()
)

district_adj_bonus_expr = quicksum(district_adj_bonus_expr_terms)

# --- Final Objective Check & Setting ---
print("\n--- Final Objective Check ---")
print(f"Base Objective Expr Type: {type(base_objective_expr)}")
print(f"District Adj Bonus Expr Type: {type(district_adj_bonus_expr)}")

# Combine objective parts
final_obj = base_objective_expr + district_adj_bonus_expr + district_base_value_expr
print(f"Final Objective Expr Type: {type(final_obj)}")
print("Setting final objective in Gurobi...")
model.setObjective(final_obj, GRB.MAXIMIZE)
print("--- End Final Objective Check ---")

print("Gurobi objective function defined.")

Calculating objective function coefficients...
Calculating base objective expression...
Calculating district adjacency bonus contribution...

--- Final Objective Check ---
Base Objective Expr Type: <class 'gurobipy._core.LinExpr'>
District Adj Bonus Expr Type: <class 'gurobipy._core.LinExpr'>
Final Objective Expr Type: <class 'gurobipy._core.LinExpr'>
Setting final objective in Gurobi...
--- End Final Objective Check ---
Gurobi objective function defined.


In [183]:
# --- Add Core Constraints ---
print("Defining core constraints...")

# 1. One District Per Workable Tile
for coords in tiles_workable:
    model.addConstr(quicksum(build_vars[coords, d] for d in ALL_DISTRICTS if (coords, d) in build_vars) <= 1,
                    name=f"OneDistPerTile_{coords[0]}_{coords[1]}")

# 2. Unique District Limit
# Ensure DISTRICTS_UNIQUE has correct, consistent names
for d in DISTRICTS_UNIQUE:
    # Check if the district 'd' is actually in the list of districts we are considering
    if d in ALL_DISTRICTS:
         model.addConstr(quicksum(build_vars[coords, d] for coords in tiles_workable if (coords, d) in build_vars) <= 1,
                         name=f"UniqueDistrict_{d}")
    # else:
    #      print(f"Warning: District '{d}' in DISTRICTS_UNIQUE but not in ALL_DISTRICTS. Skipping uniqueness constraint.")


# 3. Mutual Exclusivity
# Ensure district names in DISTRICTS_MUTUALLY_EXCLUSIVE are correct
for i, me_set in enumerate(DISTRICTS_MUTUALLY_EXCLUSIVE):
    valid_me_districts = [d for d in me_set if d in ALL_DISTRICTS] # Filter for districts actually in model
    if valid_me_districts:
        expr = quicksum(build_vars[coords, d]
                        for coords in tiles_workable
                        for d in valid_me_districts
                        if (coords, d) in build_vars) # Check key exists
        model.addConstr(expr <= 1, name=f"MutExcl_{i}")

print("Core constraints defined.")

Defining core constraints...
Core constraints defined.


In [184]:
# --- Add Specific District Placement Constraints ---
print("Defining placement constraints...")

# 4. Aqueduct Placement Requirements (Adj to CC & Freshwater)
AQUEDUCT = "Aqueduct"
if AQUEDUCT in ALL_DISTRICTS:
    for coords in tiles_workable:
        key = (coords, AQUEDUCT)
        if key in build_vars: # Check if variable exists
            # Force 0 if not adjacent to CC
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_cc.get(coords, False) else 0),
                            name=f"AqueductAdjCC_{coords[0]}_{coords[1]}")
            # Force 0 if not adjacent to Freshwater
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_freshwater.get(coords, False) else 0),
                            name=f"AqueductFreshwater_{coords[0]}_{coords[1]}")

# 5. Harbor Placement Requirements (Coast & Adj to Land)
HARBOR = "Harbor"
if HARBOR in ALL_DISTRICTS:
    for coords in tiles_workable:
        key = (coords, HARBOR)
        if key in build_vars:
            # Force 0 if not Coast
            model.addConstr(build_vars[key] <= (1 if is_coast.get(coords, False) else 0),
                            name=f"HarborIsCoast_{coords[0]}_{coords[1]}")
            # Force 0 if not adjacent to Land
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_land.get(coords, False) else 0),
                            name=f"HarborAdjLand_{coords[0]}_{coords[1]}")

# 6. Dam Placement Requirements (On Floodplains & On River)
DAM = "Dam"
if DAM in ALL_DISTRICTS:
    for coords in tiles_workable:
         key = (coords, DAM)
         if key in build_vars:
              # Force 0 if not on Floodplains tile
              model.addConstr(build_vars[key] <= (1 if is_floodplains.get(coords, False) else 0),
                              name=f"DamIsFloodplains_{coords[0]}_{coords[1]}")
              # Force 0 if tile does not have a river edge
              model.addConstr(build_vars[key] <= (1 if has_river_edge.get(coords, False) else 0),
                              name=f"DamIsRiver_{coords[0]}_{coords[1]}")

# 7. Only Harbor can be built on coast tiles
for coords in tiles_workable:
    if is_coast.get(coords, False):  # If this is a coast tile
        for d in ALL_DISTRICTS:
            if d != "Harbor":  # For all non-Harbor districts
                key = (coords, d)
                if key in build_vars:
                    # Prevent non-Harbor districts from being placed on coast
                    model.addConstr(build_vars[key] <= 0, 
                                   name=f"NoNonHarbor_OnCoast_{coords[0]}_{coords[1]}_{d}")

print("Placement constraints defined.")

Defining placement constraints...
Placement constraints defined.


In [185]:
# --- Solve the Optimization Problem ---
print("Solving the optimization problem using Gurobi...")
# Optional: Write model to file for debugging
# model.write("civ6_model_debug.lp")
# print("Model written to civ6_model_debug.lp")

model.optimize()

Solving the optimization problem using Gurobi...
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10004 rows, 3596 columns and 30204 nonzeros
Model fingerprint: 0x44256569
Variable types: 0 continuous, 3596 integer (3596 binary)
Coefficient statistics:
  Matrix range     [1e-01, 6e+00]
  Objective range  [5e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 7478 rows and 2004 columns
Presolve time: 0.10s
Presolved: 2526 rows, 1592 columns, 6079 nonzeros
Variable types: 0 continuous, 1592 integer (1592 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: objective 5.327980e+01, 2052 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj 

In [186]:
# --- Print the Optimization Results ---
print("\n--- Gurobi Optimization Results ---")

if model.Status == GRB.OPTIMAL:
    print(f"Status: Optimal")
    print(f"Optimal Total Adjacency Score = {model.ObjVal:.2f}")
    print("\nOptimal District Placements:")
    placed_districts = 0
    solution_placements = []
    # Iterate through build_vars dictionary
    for key, var in build_vars.items():
         # Use a tolerance for checking binary variable solution
         if var.X > 0.5:
              coords, district_name = key
              solution_placements.append((district_name, coords[0], coords[1]))
              placed_districts += 1

    # Sort placements for readability (by district name, then coords)
    solution_placements.sort()
    for placement in solution_placements:
        print(f"  Build {placement[0]} at ({placement[1]}, {placement[2]})")

    if placed_districts == 0:
        print("  No districts were placed in the optimal solution.")

# --- Modify this block within Cell 15 ---

# --- Modify this block within Cell 15 AGAIN ---

elif model.Status == GRB.INFEASIBLE:
    print("Status: Infeasible - The problem has no solution.")
    print("Check constraints, data (especially workable tiles), and bonus values.")
    # Compute and print IIS (Irreducible Inconsistent Subsystem) to help debug
    print("\nComputing IIS to identify conflicting constraints...")
    try:
        model.computeIIS() # Compute the IIS first
        model.update() # Ensure model state is updated after IIS computation

        # --- Get constraints using indices from IIS results ---
        print("Conflicting Constraints:")
        if hasattr(model, 'IISConstr'):
            # Get the list of all linear constraints in the model
            all_constrs = model.getConstrs()
            # IISConstr likely contains indices into this list
            iis_constr_indices = model.IISConstr
            if not iis_constr_indices:
                 print("  No conflicting constraints found in IISConstr list.")
            else:
                 for index in iis_constr_indices:
                      try:
                           # Use the index to get the actual constraint object
                           constraint = all_constrs[index]
                           print(f"  {constraint.ConstrName}")
                      except IndexError:
                           print(f"  Error: Invalid constraint index {index} found in IISConstr.")
                      except AttributeError:
                           print(f"  Error: Object at index {index} has no ConstrName.")
        else:
             print("  Could not retrieve IIS Constraints (model.IISConstr attribute missing).")


        # --- Get variable bounds using indices from IIS results ---
        print("Conflicting Variable Bounds:")
        lb_conflict = False
        ub_conflict = False
        if hasattr(model, 'IISLB') or hasattr(model, 'IISUB'):
             # Get the list of all variables in the model
             all_vars = model.getVars()

             if hasattr(model, 'IISLB'):
                  iis_lb_indices = model.IISLB
                  if iis_lb_indices:
                       lb_conflict = True
                       for index in iis_lb_indices:
                            try:
                                 variable = all_vars[index]
                                 print(f"  {variable.VarName} (Lower Bound)")
                            except IndexError:
                                 print(f"  Error: Invalid variable index {index} found in IISLB.")
                            except AttributeError:
                                 print(f"  Error: Object at index {index} has no VarName.")
                  # else: print("  No conflicting lower bounds found.") # Optional message

             if hasattr(model, 'IISUB'):
                  iis_ub_indices = model.IISUB
                  if iis_ub_indices:
                       ub_conflict = True
                       for index in iis_ub_indices:
                            try:
                                 variable = all_vars[index]
                                 print(f"  {variable.VarName} (Upper Bound)")
                            except IndexError:
                                 print(f"  Error: Invalid variable index {index} found in IISUB.")
                            except AttributeError:
                                 print(f"  Error: Object at index {index} has no VarName.")
                  # else: print("  No conflicting upper bounds found.") # Optional message

        if not lb_conflict and not ub_conflict:
             print("  No conflicting variable bounds found in IIS lists.")


        # Optional: Write IIS to a file for detailed analysis
        # model.write("civ6_infeasible.ilp")
        # print("IIS written to civ6_infeasible.ilp")

    except GurobiError as iis_error:
        print(f"Could not compute IIS or get constraints/variables: {iis_error}")
    except AttributeError as attr_error:
         # Catch potential lingering issues or version differences
         print(f"Attribute Error during IIS processing (check Gurobi version/API): {attr_error}")
    except Exception as e:
         print(f"An error occurred during IIS computation: {e}")

# --- End modification ---


elif model.Status == GRB.UNBOUNDED:
    print("Status: Unbounded - The objective can be increased indefinitely.")
    print("This usually indicates missing constraints or an error in the objective function/coefficients.")
else:
    # Consult Gurobi documentation for other status codes
    print(f"Status Code: {model.Status} (Not Optimal/Infeasible/Unbounded)")
    print("Solver did not find an optimal solution. Check Gurobi logs or status code meaning.")

print("--- End Results ---")


--- Gurobi Optimization Results ---
Status: Optimal
Optimal Total Adjacency Score = 40.50

Optimal District Placements:
  Build Aqueduct at (57, 25)
  Build Campus at (55, 27)
  Build Canal at (56, 26)
  Build Commercial Hub at (58, 26)
  Build Dam at (58, 27)
  Build Entertainment Complex at (56, 27)
  Build Government Plaza at (58, 25)
  Build Harbor at (59, 25)
  Build Holy Site at (54, 26)
  Build Industrial Zone at (57, 26)
  Build Theater Square at (55, 26)
--- End Results ---


In [187]:
# --- Debugging Checks (Run After Printing Results) ---

# print contribution of district base value to objective
if model.Status == GRB.OPTIMAL:
    district_base_value = sum(DISTRICT_BASE_VALUE.get(key[1], 0) * var.X 
                             for key, var in build_vars.items() if var.X > 0.5)
    print(f"District Base Value Contribution: {district_base_value:.2f}")

# Add after solving to check why GP isn't placed
if model.Status == GRB.OPTIMAL:
    print("District-to-district adjacency contributions:")
    for key, var in adj_district_vars.items():
        if var.X > 0.5:
            coords, d1, d2 = key
            bonus = DISTRICT_PAIR_BONUS.get((d1, d2), 0)
            print(f"  {d1} at {coords} gets +{bonus} from adjacent {d2}")
        else:
            # print(f"  {key} not placed in solution (var.X = {var.X:.2f})")
            pass

# Find optimal campus location if one was placed
optimal_campus_loc = None
if model.Status == GRB.OPTIMAL:
    for key, var in build_vars.items():
        if var.X > 0.5 and key[1] == 'Campus':
            optimal_campus_loc = key[0]
            break # Assume only one campus

if optimal_campus_loc:
    print(f"\n--- Debugging GP Adjacency to Optimal Campus at {optimal_campus_loc} ---")

    # Check if the optimal location is in the adjacency map
    if optimal_campus_loc not in adjacency_map:
        print(f"ERROR: Optimal Campus location {optimal_campus_loc} not found in adjacency_map keys!")
    else:
        adj_to_campus = adjacency_map[optimal_campus_loc]
        print(f"Tiles adjacent to Campus {optimal_campus_loc}: {adj_to_campus}")

        # Find which of these adjacent tiles are actually workable
        workable_adj_to_campus = adj_to_campus.intersection(tiles_workable)
        print(f"WORKABLE tiles adjacent to Campus: {workable_adj_to_campus}")

        if not workable_adj_to_campus:
            print("\nCONCLUSION: There are NO workable tiles adjacent to the optimal Campus location.")
            print("The Government Plaza cannot be placed adjacent to the Campus, so its bonus cannot be achieved.")
        else:
            print("\nINFO: There ARE workable tiles adjacent to the Campus where GP could potentially be placed.")
            # Further check if GP build vars exist for these specific tiles
            gp_build_vars_on_adj = []
            for adj_coords in workable_adj_to_campus:
                 gp_key = (adj_coords, 'Government Plaza')
                 if gp_key in build_vars:
                      gp_build_vars_on_adj.append(build_vars[gp_key].VarName)
                 else:
                      print(f"  WARNING: Build variable for GP not found for workable adjacent tile {adj_coords}!")
            if gp_build_vars_on_adj:
                 print(f"  GP build variables exist for adjacent workable tiles: {len(gp_build_vars_on_adj)} found.")

    # Verify the specific auxiliary variable exists and the bonus value is read correctly
    adj_var_key = (optimal_campus_loc, 'Campus', 'Government Plaza')
    if adj_var_key in adj_district_vars:
         print(f"Auxiliary variable for Campus <- GP bonus exists: {adj_district_vars[adj_var_key].VarName}")
         # Use the *actual* dictionary value used in the objective calculation
         bonus_val = DISTRICT_PAIR_BONUS.get(('Campus', 'Government Plaza'), -999) # Use -999 to show if key missing
         print(f"Bonus value ('Campus', 'Government Plaza') read from dict: {bonus_val}")
         # Check if the variable was active in the solution (if GP bonus was expected)
         if model.Status == GRB.OPTIMAL:
             print(f"  Value of Aux Var in solution: {adj_district_vars[adj_var_key].X:.0f}")

    else:
         print(f"ERROR: Auxiliary variable {adj_var_key} DOES NOT exist. Check its creation loop.")

    print("--- End GP Debug ---")
elif model.Status == GRB.OPTIMAL:
    print("\n--- GP Debug Skipped: No Campus placed in optimal solution. ---")

District Base Value Contribution: 15.00
District-to-district adjacency contributions:
  Commercial Hub at (58, 26) gets +2 from adjacent Harbor
  Theater Square at (55, 26) gets +1 from adjacent Entertainment Complex
  Industrial Zone at (57, 26) gets +2 from adjacent Aqueduct
  Industrial Zone at (57, 26) gets +2 from adjacent Dam
  Industrial Zone at (57, 26) gets +2 from adjacent Canal
  Harbor at (59, 25) gets +1 from adjacent Government Plaza
  Commercial Hub at (58, 26) gets +1 from adjacent Government Plaza
  Industrial Zone at (57, 26) gets +1 from adjacent Government Plaza
  Campus at (55, 27) gets +0.5 from adjacent Holy Site
  Campus at (55, 27) gets +0.5 from adjacent Theater Square
  Campus at (55, 27) gets +0.5 from adjacent Entertainment Complex
  Holy Site at (54, 26) gets +0.5 from adjacent Campus
  Holy Site at (54, 26) gets +0.5 from adjacent Theater Square
  Theater Square at (55, 26) gets +0.5 from adjacent Campus
  Theater Square at (55, 26) gets +0.5 from adjacen

In [188]:
def update_json_with_district_recommendations(build_vars, city_center_coords, 
                                             data_file='civ_map_data.json',
                                             scenario_name=None):
    # Load existing data
    with open(data_file, 'r') as f:
        data = json.load(f)
    
    # Track which tiles received district recommendations
    tiles_with_districts = {}
    
    # Add city center to the recommendations
    x_cc, y_cc = city_center_coords
    cc_key = f"{x_cc},{y_cc}"
    tiles_with_districts[cc_key] = "citycenter"
    
    # Extract district placements from optimization results
    for (coords, district_type), var in build_vars.items():
        if var.X > 0.5:
            x, y = coords
            key = f"{x},{y}"
            formatted_district = district_type.lower().replace(' ', '')
            tiles_with_districts[key] = formatted_district
    
    # Determine field name based on scenario
    field_name = 'recommended_district'
    if scenario_name:
        field_name = f'scenario_{scenario_name}'
    
    # Update tiles
    recommended_count = 0
    
    for tile in data['tiles']:
        key = f"{tile['x']},{tile['y']}"
        
        # Clear previous data for this specific scenario
        if field_name in tile:
            del tile[field_name]
            
        # Add new recommendation if this tile has one
        if key in tiles_with_districts:
            tile[field_name] = tiles_with_districts[key]
            recommended_count += 1
    
    # Write the updated data back
    with open(data_file, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Updated JSON: {recommended_count} tiles with {field_name}")

update_json_with_district_recommendations(build_vars, (cc_x, cc_y))

Updated JSON: 12 tiles with recommended_district


In [189]:
# SENSITIVITY ANALYSIS
def run_district_optimization(district_values=None, scenario_name=None):
    """Run the optimization with custom district values
    
    Args:
        district_values: Dictionary mapping districts to their base values
        scenario_name: Name for this optimization scenario
        
    Returns:
        model: The optimized Gurobi model
        build_vars: Dictionary of decision variables
        results: Dictionary with optimization results
    """
    global DISTRICT_BASE_VALUE  # Reference the global constant
    
    # Save original values to restore later
    original_values = DISTRICT_BASE_VALUE.copy()
    
    try:
        # Use provided values if available
        if district_values:
            # Update base values with provided priorities
            for district, value in district_values.items():
                DISTRICT_BASE_VALUE[district] = value
        
        # --- Initialize model ---
        model = Model("Civ6_District_Optimization")
        model.setParam('OutputFlag', 0)  # Suppress output for batch runs
        
        # --- Define Main Decision Variables: Build(x, y, d) ---
        build_vars = {}
        print("Creating main 'Build' variables...")
        for coords in tiles_workable:
            for d in ALL_DISTRICTS:
                var_name = f"Build[{coords[0]},{coords[1]},{d}]"
                build_vars[coords, d] = model.addVar(vtype=GRB.BINARY, name=var_name)

        # Integrate new variables immediately (good practice)
        model.update()
        print(f"Gurobi 'Build' variables defined ({len(build_vars)} variables).")

        # --- Define Auxiliary Variables for District Adjacency ---
        adj_district_vars = {} # Format: adj_district_vars[coords_d1, d1, d2] = variable A

        print("Defining auxiliary adjacency variables...")
        # Create variables only for pairs defined in DISTRICT_PAIR_BONUS where d1 receives bonus from d2
        for (d1, d2), bonus_value in DISTRICT_PAIR_BONUS.items():
            if bonus_value > 0:
                # Check if d2 is a district type or a feature like 'Wonder'
                is_d2_district = d2 in ALL_DISTRICTS
                is_d2_feature = d2 in FEATURES # e.g., 'Wonder' if handled as feature

                for coords_d1 in tiles_workable:
                    # Ensure the primary district (d1) variable exists for this tile
                    if (coords_d1, d1) in build_vars:
                        # Create the aux variable: AdjDist_x_y_d1_from_d2 = 1 if d1 at (x,y) gets bonus from adjacent d2
                        var_name = f"AdjDist_{coords_d1[0]}_{coords_d1[1]}_{d1}_from_{d2}"
                        # Use tuple key: (coords_tuple, d1_string, d2_string)
                        current_key = (coords_d1, d1, d2) # Define the key explicitly
                        adj_district_vars[current_key] = model.addVar(vtype=GRB.BINARY, name=var_name)

                        # --- Add specific debug print for variable creation ---
                        if coords_d1 == (51, 16) and d1 == 'Campus' and d2 == 'Government Plaza':
                            print(f"\n*** DEBUG: SUCCESSFULLY CREATED aux var for key {current_key} with name {var_name} ***\n")
                        # --- End debug print ---


        # Integrate new variables
        model.update()
        print(f"Defined {len(adj_district_vars)} auxiliary adjacency variables.")
        # Check if the specific key exists AFTER the loop
        check_key = ((51, 16), 'Campus', 'Government Plaza')
        if check_key in adj_district_vars:
            print(f"DEBUG: Key {check_key} EXISTS in adj_district_vars after creation loop.")
        else:
            print(f"*** CRITICAL WARNING: Key {check_key} DOES NOT EXIST in adj_district_vars after creation loop! ***")

        # --- Add Linking Constraints for Auxiliary Adjacency Variables ---
        # Links A = adj_district_vars[coords_d1, d1, d2]
        # To B1 = build_vars[coords_d1, d1]
        # And S2 = Sum over adjacent tiles (adj_coords) of Build(adj_coords, d2) or HasFeature(adj_coords, d2)

        print("Adding linking constraints for adjacency variables...")
        constraints_added_count = 0

        for (coords_d1, d1, d2), A in adj_district_vars.items():
            # Get the B1 variable (Build(d1) at coords_d1)
            if (coords_d1, d1) not in build_vars: continue # Should not happen based on creation logic, but safe check
            B1 = build_vars[coords_d1, d1]

            # Calculate S2 = Sum of adjacent triggers (d2)
            adjacent_tiles = adjacency_map.get(coords_d1, set())
            sum_expr_S2 = None # Initialize S2 expression

            # Determine if d2 is a District or a Feature triggering the bonus
            if d2 in ALL_DISTRICTS:
                # S2 is the sum of Build(adj_coords, d2) for adjacent workable tiles
                # Use a list comprehension inside quicksum for clarity if needed
                s2_terms = [build_vars.get((adj_coords, d2), 0)
                            for adj_coords in adjacent_tiles
                            if adj_coords in tiles_workable and (adj_coords, d2) in build_vars] # Ensure var exists
                sum_expr_S2 = quicksum(s2_terms)

            elif d2 in FEATURES: # Check if d2 is a key in FEATURES dict (e.g., "Wonder")
                # S2 is the sum of indicator constants (1 if feature present, 0 otherwise)
                sum_expr_S2 = quicksum(1
                                        for adj_coords in adjacent_tiles
                                        if adj_coords in features_on_tile and d2 in features_on_tile[adj_coords])
            else:
                # Skip if d2 is not a district or known feature key
                continue

            # --- DEBUG PRINT for S2 expression (Now Included) ---
            if d1 == 'Campus' and d2 == 'Government Plaza' and coords_d1 == (51, 16): # Check for specific Campus location
                print(f"\n*** DEBUG constraint S2 check for Campus at {coords_d1} from GP: ***")
                try:
                    # Check if S2 is a valid Gurobi expression type
                    if isinstance(sum_expr_S2, (int, float, type(None))): # Check if it resolved to a constant or None
                        print(f"  S2 Expression evaluated to constant/None: {sum_expr_S2}")
                    else: # Assume it's a Gurobi LinExpr
                        print(f"  S2 Expression (Gurobi): {sum_expr_S2}")

                    # Check components manually to see which vars S2 depends on
                    terms = []
                    s2_term_vars_found = False
                    for adj_coords_debug in adjacency_map.get(coords_d1, set()):
                            if adj_coords_debug in tiles_workable:
                                key_debug = (adj_coords_debug, 'Government Plaza')
                                if key_debug in build_vars:
                                    terms.append(build_vars[key_debug].VarName)
                                    s2_term_vars_found = True
                    print(f"  S2 depends on GP Build Vars on workable adjacent tiles: {terms if s2_term_vars_found else '*** NONE FOUND - THIS IS LIKELY THE PROBLEM ***'}")
                except Exception as e:
                    print(f"  Error printing S2 expression/components: {e}")
                print("*** End S2 Check ***\n")
            # --- End S2 Debug Print ---


            # --- Constraints linking A to B1 and S2: A = 1 iff B1=1 and S2>=1 ---
            constr_name_base = f"LinkAdj_{coords_d1[0]}_{coords_d1[1]}_{d1}_{d2}"

            # Check if sum_expr_S2 is valid before adding constraints using it
            if sum_expr_S2 is not None:
                # 1. A must be 0 or 1
                model.addConstr(A <= 1)
                
                # 2. A can only be 1 if B1 is 1
                model.addConstr(A <= B1)
                
                # 3. Create indicator for adjacency
                has_adj = model.addVar(vtype=GRB.BINARY, name=f"{constr_name_base}_HasAdj")
                model.addConstr(sum_expr_S2 <= MAX_ADJACENT_TILES * has_adj)
                model.addConstr(has_adj * 0.1 <= sum_expr_S2)
                
                # 4. A can only be 1 if there's adjacency
                model.addConstr(A <= has_adj)
                
                # 5. A must be 1 if both B1 and has_adj are 1
                model.addConstr(A >= B1 + has_adj - 1)

            # else:
            #      print(f"Warning: sum_expr_S2 was None for {constr_name_base}. Constraints not added") # Optional


        print(f"Added {constraints_added_count} linking constraints for adjacency.")

        # --- Define Objective Function ---

        print("Calculating objective function coefficients...")

        # Calculate base coefficients from Terrain, Features (non-district), CC, River
        obj_coeffs = {}
        for coords in tiles_workable:
            for d in ALL_DISTRICTS:
                # Sum terrain/feature bonuses from adjacent tiles (non-River)
                terrain_feature_bonus = 0
                for adj_coords in adjacency_map.get(coords, set()):
                    if adj_coords in features_on_tile:
                        for f in features_on_tile.get(adj_coords, set()):
                            # Look up bonus value, ensuring f isn't the River key used for edge logic
                            if f != RIVER_FEATURE_KEY:
                                terrain_feature_bonus += TERRAIN_BONUS_VALUE.get((d, f), 0)

                # Add City Center bonus if adjacent
                cc_bonus = CC_BONUS_VALUE.get(d, 0) if is_adjacent_to_cc.get(coords, False) else 0

                # Add River Adjacency Bonus if *this tile* has a river edge
                river_bonus = 0
                if has_river_edge.get(coords, False):
                    river_bonus = TERRAIN_BONUS_VALUE.get((d, RIVER_FEATURE_KEY), 0)

                obj_coeffs[coords, d] = terrain_feature_bonus + cc_bonus + river_bonus

        print("Calculating base objective expression...")
        # Base objective part from terrain, CC, river bonuses
        base_objective_expr = quicksum(
            build_vars[key] * obj_coeffs.get(key, 0)
            for key in build_vars.keys() # Iterate through all defined build variables
        )

        print("Calculating district adjacency bonus contribution...")
        # District adjacency bonus part using the auxiliary variables
        district_adj_bonus_expr_terms = []
        processed_obj_keys = set() # Keep track of keys processed in this loop

        for key, aux_var in adj_district_vars.items():
            # Ensure key is the expected tuple format (coords_tuple, d1_string, d2_string)
            if not (isinstance(key, tuple) and len(key) == 3):
                print(f"WARNING: Unexpected key format in adj_district_vars: {key}")
                continue
            coords_d1, d1, d2 = key # Unpack the key
            processed_obj_keys.add(key) # Mark this key as processed

            bonus = DISTRICT_PAIR_BONUS.get((d1, d2), 0)
            if bonus > 0:
                term = aux_var * bonus
                district_adj_bonus_expr_terms.append(term)

                # --- Add specific debug print for GP bonus ---
                if coords_d1 == (51, 16) and d1 == 'Campus' and d2 == 'Government Plaza':
                    print(f"\n*** DEBUG: FOUND AND ADDING objective term for {aux_var.VarName} * {bonus} (GP->Campus Bonus) ***\n")
                # --- End debug print ---

        # Check if the specific key we care about was missed in the loop
        check_key_obj = ((51, 16), 'Campus', 'Government Plaza')
        if check_key_obj in adj_district_vars and check_key_obj not in processed_obj_keys:
            print(f"*** CRITICAL WARNING: Key {check_key_obj} exists in adj_district_vars but WAS NOT processed in objective loop! ***")


        # District base value contribution 
        district_base_value_expr = quicksum(
            build_vars[key] * DISTRICT_BASE_VALUE.get(key[1], 0.0)  # key[1] is the district type
            for key in build_vars.keys()
        )

        district_adj_bonus_expr = quicksum(district_adj_bonus_expr_terms)

        # --- Final Objective Check & Setting ---
        print("\n--- Final Objective Check ---")
        print(f"Base Objective Expr Type: {type(base_objective_expr)}")
        print(f"District Adj Bonus Expr Type: {type(district_adj_bonus_expr)}")

        # Combine objective parts
        final_obj = base_objective_expr + district_adj_bonus_expr + district_base_value_expr
        print(f"Final Objective Expr Type: {type(final_obj)}")
        print("Setting final objective in Gurobi...")
        model.setObjective(final_obj, GRB.MAXIMIZE)
        print("--- End Final Objective Check ---")

        print("Gurobi objective function defined.")

        # --- Add Core Constraints ---
        print("Defining core constraints...")

        # 1. One District Per Workable Tile
        for coords in tiles_workable:
            model.addConstr(quicksum(build_vars[coords, d] for d in ALL_DISTRICTS if (coords, d) in build_vars) <= 1,
                            name=f"OneDistPerTile_{coords[0]}_{coords[1]}")

        # 2. Unique District Limit
        # Ensure DISTRICTS_UNIQUE has correct, consistent names
        for d in DISTRICTS_UNIQUE:
            # Check if the district 'd' is actually in the list of districts we are considering
            if d in ALL_DISTRICTS:
                model.addConstr(quicksum(build_vars[coords, d] for coords in tiles_workable if (coords, d) in build_vars) <= 1,
                                name=f"UniqueDistrict_{d}")
            # else:
            #      print(f"Warning: District '{d}' in DISTRICTS_UNIQUE but not in ALL_DISTRICTS. Skipping uniqueness constraint.")


        # 3. Mutual Exclusivity
        # Ensure district names in DISTRICTS_MUTUALLY_EXCLUSIVE are correct
        for i, me_set in enumerate(DISTRICTS_MUTUALLY_EXCLUSIVE):
            valid_me_districts = [d for d in me_set if d in ALL_DISTRICTS] # Filter for districts actually in model
            if valid_me_districts:
                expr = quicksum(build_vars[coords, d]
                                for coords in tiles_workable
                                for d in valid_me_districts
                                if (coords, d) in build_vars) # Check key exists
                model.addConstr(expr <= 1, name=f"MutExcl_{i}")

        print("Core constraints defined.")
        
        # --- Add Specific District Placement Constraints ---
        print("Defining placement constraints...")

        # 4. Aqueduct Placement Requirements (Adj to CC & Freshwater)
        AQUEDUCT = "Aqueduct"
        if AQUEDUCT in ALL_DISTRICTS:
            for coords in tiles_workable:
                key = (coords, AQUEDUCT)
                if key in build_vars: # Check if variable exists
                    # Force 0 if not adjacent to CC
                    model.addConstr(build_vars[key] <= (1 if is_adjacent_to_cc.get(coords, False) else 0),
                                    name=f"AqueductAdjCC_{coords[0]}_{coords[1]}")
                    # Force 0 if not adjacent to Freshwater
                    model.addConstr(build_vars[key] <= (1 if is_adjacent_to_freshwater.get(coords, False) else 0),
                                    name=f"AqueductFreshwater_{coords[0]}_{coords[1]}")

        # 5. Harbor Placement Requirements (Coast & Adj to Land)
        HARBOR = "Harbor"
        if HARBOR in ALL_DISTRICTS:
            for coords in tiles_workable:
                key = (coords, HARBOR)
                if key in build_vars:
                    # Force 0 if not Coast
                    model.addConstr(build_vars[key] <= (1 if is_coast.get(coords, False) else 0),
                                    name=f"HarborIsCoast_{coords[0]}_{coords[1]}")
                    # Force 0 if not adjacent to Land
                    model.addConstr(build_vars[key] <= (1 if is_adjacent_to_land.get(coords, False) else 0),
                                    name=f"HarborAdjLand_{coords[0]}_{coords[1]}")

        # 6. Dam Placement Requirements (On Floodplains & On River)
        DAM = "Dam"
        if DAM in ALL_DISTRICTS:
            for coords in tiles_workable:
                key = (coords, DAM)
                if key in build_vars:
                    # Force 0 if not on Floodplains tile
                    model.addConstr(build_vars[key] <= (1 if is_floodplains.get(coords, False) else 0),
                                    name=f"DamIsFloodplains_{coords[0]}_{coords[1]}")
                    # Force 0 if tile does not have a river edge
                    model.addConstr(build_vars[key] <= (1 if has_river_edge.get(coords, False) else 0),
                                    name=f"DamIsRiver_{coords[0]}_{coords[1]}")

        # 7. Only Harbor can be built on coast tiles
        for coords in tiles_workable:
            if is_coast.get(coords, False):  # If this is a coast tile
                for d in ALL_DISTRICTS:
                    if d != "Harbor":  # For all non-Harbor districts
                        key = (coords, d)
                        if key in build_vars:
                            # Prevent non-Harbor districts from being placed on coast
                            model.addConstr(build_vars[key] <= 0, 
                                        name=f"NoNonHarbor_OnCoast_{coords[0]}_{coords[1]}_{d}")

        print("Placement constraints defined.")

        # Run optimization
        model.optimize()
        
        # Collect results
        results = {
            "objective_value": model.ObjVal if model.Status == GRB.OPTIMAL else None,
            "status": model.Status,
            "district_placements": [],
            "scenario": scenario_name
        }
        
        if model.Status == GRB.OPTIMAL:
            for key, var in build_vars.items():
                if var.X > 0.5:
                    coords, district_name = key
                    results["district_placements"].append((district_name, coords))
            
            # Update JSON with this scenario
            update_json_with_district_recommendations(
                build_vars, 
                city_center_coords, 
                scenario_name=scenario_name
            )
            
        return model, build_vars, results
        
    finally:
        # Restore original values
        DISTRICT_BASE_VALUE = original_values

In [190]:
def run_yield_sensitivity_analysis(yields_to_analyze=None, boost_factors=None):
    """Run optimization for different yield priorities
    
    Args:
        yields_to_analyze: List of yields to prioritize
        boost_factors: List of boost factors to try for each yield
        
    Returns:
        Dictionary of results by yield and boost factor
    """
    if yields_to_analyze is None:
        yields_to_analyze = ["science", "culture", "production", "gold", "faith"]
    
    if boost_factors is None:
        boost_factors = [1.5, 2.0, 3.0]
        
    # Also run a balanced scenario
    results = {
        "balanced": {}
    }
    
    # Run balanced scenario first
    print(f"Running balanced scenario...")
    _, _, balanced_result = run_district_optimization(
        district_values=DISTRICT_BASE_VALUE,
        scenario_name="balanced"
    )
    results["balanced"] = balanced_result
    
    # Run scenarios for each yield and boost factor
    for yield_type in yields_to_analyze:
        results[yield_type] = {}
        
        for boost in boost_factors:
            scenario_name = f"{yield_type}_{boost}"
            print(f"Running {scenario_name} scenario...")
            
            # Create priority profile
            priority_profile = create_priority_profile(yield_type, boost)
            
            # Run optimization with these priorities
            _, _, scenario_result = run_district_optimization(
                district_values=priority_profile,
                scenario_name=scenario_name
            )
            
            # Store results
            results[yield_type][boost] = scenario_result
            
    return results

In [191]:
def visualize_yield_sensitivity_results(results):
    """Create visualizations for yield sensitivity analysis
    
    Args:
        results: Dictionary from run_yield_sensitivity_analysis
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Extract data for visualization
    yields = [y for y in results.keys() if y != "balanced"]
    boost_factors = list(results[yields[0]].keys())
    
    # Prepare data for objective value comparison
    obj_values = []
    for yield_type in yields:
        yield_values = [results[yield_type][b]["objective_value"] for b in boost_factors]
        obj_values.append(yield_values)
    
    balanced_obj = results["balanced"]["objective_value"]
    
    # 1. Objective Value Comparison
    plt.figure(figsize=(12, 8))
    x = np.arange(len(boost_factors))
    width = 0.15
    offsets = np.linspace(-0.3, 0.3, len(yields))
    
    for i, yield_type in enumerate(yields):
        plt.bar(x + offsets[i], obj_values[i], width, label=yield_type.capitalize())
        
    plt.axhline(y=balanced_obj, color='k', linestyle='--', label='Balanced')
    
    plt.xlabel('Priority Boost Factor')
    plt.ylabel('Objective Value')
    plt.title('Objective Value by Yield Priority')
    plt.xticks(x, boost_factors)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.savefig('yield_priority_objval.png')
    plt.show()
    
    # 2. District Count Comparison
    plt.figure(figsize=(14, 8))
    
    # Count districts for each scenario
    district_counts = {}
    for yield_type in yields:
        district_counts[yield_type] = {}
        for boost in boost_factors:
            placements = results[yield_type][boost]["district_placements"]
            counts = {}
            for district, _ in placements:
                counts[district] = counts.get(district, 0) + 1
            district_counts[yield_type][boost] = counts
    
    # Count balanced districts
    balanced_counts = {}
    for district, _ in results["balanced"]["district_placements"]:
        balanced_counts[district] = balanced_counts.get(district, 0) + 1
        
    # Get unique districts across all scenarios
    all_districts = set()
    for y in district_counts:
        for b in district_counts[y]:
            all_districts.update(district_counts[y][b].keys())
    all_districts = sorted(all_districts)
    
    # Create subplots for each yield type
    fig, axs = plt.subplots(len(yields), 1, figsize=(12, 4*len(yields)))
    
    for i, yield_type in enumerate(yields):
        ax = axs[i]
        district_data = []
        
        for district in all_districts:
            district_data.append([
                district_counts[yield_type][b].get(district, 0) - balanced_counts.get(district, 0)
                for b in boost_factors
            ])
        
        x = np.arange(len(boost_factors))
        width = 0.8 / len(all_districts)
        offsets = np.linspace(-0.4, 0.4, len(all_districts))
        
        for j, district in enumerate(all_districts):
            ax.bar(x + offsets[j], district_data[j], width, label=district)
            
        ax.set_title(f'{yield_type.capitalize()} Priority: District Count Change from Balanced')
        ax.set_ylabel('Count Difference')
        ax.set_xticks(x)
        ax.set_xticklabels(boost_factors)
        ax.legend(ncol=3)
        ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('yield_priority_districts.png')
    plt.show()

In [192]:
def analyze_district_adjacency_patterns(results_dict, adjacency_map):
    """Analyze district adjacency patterns across different playstyles
    
    Args:
        results_dict: Dictionary of sensitivity analysis results 
                      (from run_yield_sensitivity_analysis)
        adjacency_map: Your existing map of tile adjacencies
        
    Returns:
        Dictionary with adjacency analysis results
    """
    import numpy as np
    
    # Get all unique district types across all scenarios
    all_districts = set()
    
    # Handle balanced scenario
    if "balanced" in results_dict and isinstance(results_dict["balanced"], dict) and "district_placements" in results_dict["balanced"]:
        for district, _ in results_dict["balanced"]["district_placements"]:
            all_districts.add(district)
    
    # Handle yield scenarios
    for yield_type in results_dict:
        if yield_type == "balanced":
            continue
        
        # Skip if not a dictionary
        if not isinstance(results_dict[yield_type], dict):
            continue
            
        for boost_factor in results_dict[yield_type]:
            # Skip if not a dictionary
            if not isinstance(results_dict[yield_type][boost_factor], dict):
                continue
                
            if "district_placements" in results_dict[yield_type][boost_factor]:
                for district, _ in results_dict[yield_type][boost_factor]["district_placements"]:
                    all_districts.add(district)
    
    all_districts = sorted(list(all_districts))
    n_districts = len(all_districts)
    
    # Dictionary to store adjacency matrices for each scenario
    adjacency_matrices = {}
    
    # Process balanced scenario
    if "balanced" in results_dict and isinstance(results_dict["balanced"], dict) and "district_placements" in results_dict["balanced"]:
        placements = results_dict["balanced"]["district_placements"]
        scenario_name = "balanced"
        
        # Create map of coordinates to district type
        coord_to_district = {coords: district for district, coords in placements}
        
        # Initialize adjacency matrix
        adj_matrix = np.zeros((n_districts, n_districts))
        
        # Count district adjacencies
        for district1, coords1 in placements:
            idx1 = all_districts.index(district1)
            
            # Check all adjacent tiles
            for adj_coords in adjacency_map.get(coords1, set()):
                if adj_coords in coord_to_district:
                    district2 = coord_to_district[adj_coords]
                    idx2 = all_districts.index(district2)
                    
                    # Increment adjacency count
                    adj_matrix[idx1, idx2] += 1
        
        adjacency_matrices[scenario_name] = adj_matrix
    
    # Process yield scenarios
    for yield_type in results_dict:
        if yield_type == "balanced":
            continue
        
        # Skip if not a dictionary
        if not isinstance(results_dict[yield_type], dict):
            continue
            
        for boost_factor in results_dict[yield_type]:
            # Skip if not a dictionary
            if not isinstance(results_dict[yield_type][boost_factor], dict):
                continue
                
            if "district_placements" in results_dict[yield_type][boost_factor]:
                placements = results_dict[yield_type][boost_factor]["district_placements"]
                scenario_name = f"{yield_type}_{boost_factor}"
                
                # Create map of coordinates to district type
                coord_to_district = {coords: district for district, coords in placements}
                
                # Initialize adjacency matrix
                adj_matrix = np.zeros((n_districts, n_districts))
                
                # Count district adjacencies
                for district1, coords1 in placements:
                    idx1 = all_districts.index(district1)
                    
                    # Check all adjacent tiles
                    for adj_coords in adjacency_map.get(coords1, set()):
                        if adj_coords in coord_to_district:
                            district2 = coord_to_district[adj_coords]
                            idx2 = all_districts.index(district2)
                            
                            # Increment adjacency count
                            adj_matrix[idx1, idx2] += 1
                
                adjacency_matrices[scenario_name] = adj_matrix
    
    # Calculate consistency metrics across scenarios
    if not adjacency_matrices:
        raise ValueError("No valid adjacency matrices could be created")
        
    matrix_list = list(adjacency_matrices.values())
    avg_adjacency = np.mean(matrix_list, axis=0)
    var_adjacency = np.var(matrix_list, axis=0)
    
    # Identify the most frequent district pairs
    frequent_pairs = []
    for i in range(n_districts):
        for j in range(i+1, n_districts):
            if avg_adjacency[i, j] > 0:
                frequent_pairs.append({
                    "district1": all_districts[i],
                    "district2": all_districts[j],
                    "avg_frequency": avg_adjacency[i, j],
                    "variance": var_adjacency[i, j],
                    "coefficient_of_variation": np.sqrt(var_adjacency[i, j])/max(avg_adjacency[i, j], 0.0001)
                })
    
    # Sort by average frequency (descending)
    frequent_pairs.sort(key=lambda x: x["avg_frequency"], reverse=True)
    
    return {
        "district_types": all_districts,
        "adjacency_matrices": adjacency_matrices,
        "avg_adjacency": avg_adjacency,
        "var_adjacency": var_adjacency,
        "frequent_pairs": frequent_pairs
    }

In [193]:
def visualize_district_adjacency_patterns(adjacency_data):
    """Create visualizations for district adjacency patterns
    
    Args:
        adjacency_data: Output from analyze_district_adjacency_patterns
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    
    districts = adjacency_data["district_types"]
    avg_adjacency = adjacency_data["avg_adjacency"]
    var_adjacency = adjacency_data["var_adjacency"]
    
    # 1. Average Adjacency Frequency Heatmap
    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(avg_adjacency, annot=True, fmt=".1f", cmap="YlGnBu", 
                xticklabels=districts, yticklabels=districts, ax=ax)
    ax.set_title("Average District Adjacency Frequency")
    plt.tight_layout()
    plt.savefig('district_adjacency_avg.png')
    plt.close()
    
    # 2. Adjacency Variability Heatmap - shows which pairings change most with playstyle
    fig, ax = plt.subplots(figsize=(12, 10))
    # Use coefficient of variation (std/mean) to show relative variability
    cv_matrix = np.zeros_like(avg_adjacency)
    for i in range(len(districts)):
        for j in range(len(districts)):
            if avg_adjacency[i,j] > 0:
                cv_matrix[i,j] = np.sqrt(var_adjacency[i,j])/avg_adjacency[i,j]
            
    sns.heatmap(cv_matrix, annot=True, fmt=".2f", cmap="Reds", 
                xticklabels=districts, yticklabels=districts, ax=ax)
    ax.set_title("District Adjacency Variability Across Playstyles\n(Higher = More Playstyle Dependent)")
    plt.tight_layout()
    plt.savefig('district_adjacency_var.png')
    plt.close()
    
    # 3. Top District Pairs Bar Chart
    if "frequent_pairs" in adjacency_data and adjacency_data["frequent_pairs"]:
        frequent_pairs = adjacency_data["frequent_pairs"][:10]  # Top 10 most frequent pairs
        
        fig, ax = plt.subplots(figsize=(14, 8))
        # Sort pairs by coefficient of variation for this chart
        frequent_pairs.sort(key=lambda x: x["coefficient_of_variation"])
        
        labels = [f"{p['district1']}-{p['district2']}" for p in frequent_pairs]
        values = [p["avg_frequency"] for p in frequent_pairs]
        errors = [np.sqrt(p["variance"]) for p in frequent_pairs]
        cv_values = [p["coefficient_of_variation"] for p in frequent_pairs]
        
        # Use a consistent color scale
        norm = plt.Normalize(min(cv_values), max(cv_values))
        colors = plt.cm.RdYlGn_r(norm(cv_values))
        
        bars = ax.bar(range(len(frequent_pairs)), values, yerr=errors, capsize=5)
        
        # Color bars by coefficient of variation (playstyle sensitivity)
        for bar, color in zip(bars, colors):
            bar.set_color(color)
        
        ax.set_xticks(range(len(frequent_pairs)))
        ax.set_xticklabels(labels, rotation=45, ha='right')
        ax.set_ylabel("Average Adjacency Frequency")
        ax.set_title("Top District Pairs by Frequency\n(Color: Red = High Variability, Green = Consistent Across Playstyles)")
        
        # Add color bar to show scale - properly connected to the axis
        sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlGn_r, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label("Playstyle Sensitivity")
        
        plt.tight_layout()
        plt.savefig('district_adjacency_top_pairs.png')
        plt.close()
    else:
        print("Warning: No frequent pairs data available for visualization")

In [194]:
# After running your sensitivity analysis
yield_results = run_yield_sensitivity_analysis(
    yields_to_analyze=["science", "culture", "production", "gold", "faith"],
    boost_factors=[1.5, 2.0, 3.0]
)

# Analyze district adjacency patterns
adjacency_analysis = analyze_district_adjacency_patterns(yield_results, adjacency_map)

# Visualize the results
visualize_district_adjacency_patterns(adjacency_analysis)

# Print the most consistent district pairings (lowest coefficient of variation)
sorted_by_consistency = sorted(adjacency_analysis["frequent_pairs"], 
                              key=lambda x: x["coefficient_of_variation"])
print("Most consistent district pairings across playstyles:")
for pair in sorted_by_consistency[:5]:
    print(f"{pair['district1']} - {pair['district2']}: {pair['avg_frequency']:.1f} (CV: {pair['coefficient_of_variation']:.2f})")

Running balanced scenario...
Creating main 'Build' variables...
Gurobi 'Build' variables defined (348 variables).
Defining auxiliary adjacency variables...
Defined 1624 auxiliary adjacency variables.
*** CRITICAL WARNING: Key ((51, 16), 'Campus', 'Government Plaza') DOES NOT EXIST in adj_district_vars after creation loop! ***
Adding linking constraints for adjacency variables...
Added 0 linking constraints for adjacency.
Calculating objective function coefficients...
Calculating base objective expression...
Calculating district adjacency bonus contribution...

--- Final Objective Check ---
Base Objective Expr Type: <class 'gurobipy._core.LinExpr'>
District Adj Bonus Expr Type: <class 'gurobipy._core.LinExpr'>
Final Objective Expr Type: <class 'gurobipy._core.LinExpr'>
Setting final objective in Gurobi...
--- End Final Objective Check ---
Gurobi objective function defined.
Defining core constraints...
Core constraints defined.
Defining placement constraints...
Placement constraints defi